In [10]:
import numpy as np
from astropy.io import fits
def suzaku_norm(flux, area):
    return flux/area*400*3.14/5.243e-9
# suzaku_norm((67.88+35.06+67.88+153.01)*1e-15,7.23+6.28+7.23+40.03) 222_4_5
suzaku_norm((99.18+83.47+36.95)*1e-15, 15.75+13.84+5.89)

0.0015048179471970277

convert norm to em

In [1]:

import numpy as np
# 230316
# norm = 5.24274E-03 
# normerr = 7.20886E-04
# d_ang = 3.427 #kpc/arcsec at z=0.21
# kpc2cm = 3.08568e21
# area = 1.6**2*np.pi #arcmin2

# 230327
area = 3.5*4.5 #arcmin2
kpc2cm = 3.08568e21
dens = [5.81e-3, 3.13e-3, 8.14e-3]
dens_err = [4.69e-4, 4.66e-4, 5.08e-4]
d_ang = 3.427 #kpc/arcsec at z=0.21

def norm2em(norm, normerr):
    const = 1e-14/4/np.pi/(d_ang*20*60*kpc2cm*(1+0.21))**2/400/np.pi*area
    return norm/const, normerr/const
[norm2em(norm, normerr) for norm, normerr in zip(dens, dens_err)]

[(1.3733414090725391e+65, 1.1086008964802424e+64),
 (7.398551825124005e+64, 1.1015096327500916e+64),
 (1.9240962254475848e+65, 1.2007873249722029e+64)]

from srcflux file read src fluxes

In [ ]:
# plot chandra cxc pointsrc from srcflux
workpath = '/Users/eusracenorth/Documents/suzaku_A222-223/chandra/data/wavdetect'
file = open(f'{workpath}/srcflux_summary_idx14.txt')
# 0.5-7keV to 2-8keV
flux = []
dat = file.readlines()
file.close()
f, flo, fhi = [],[],[]
cts = 0
for line in dat:
    if 'Unabs Mod.Flux ' in line:
        string = line.split('Flux ')[-1].split('erg')[0].replace(" ","")
        errstr = line.split('(')[-1]
        flo.append(float(errstr.split(',')[0]))
        fhi.append(float(errstr.split(',')[-1].split(')')[0]))
        if 'E' in string:
            f.append(float(string))
        else:
            f.append(np.nan)

flo = np.array(flo)
fhi = np.array(fhi)
f = np.array(f)
flo[~np.isfinite(f)] = 0
fhi[~np.isfinite(f)] = 0
f[~np.isfinite(f)] = 0

print(f,fhi,flo)
print(np.sum(fhi>1e-14))

In [ ]:
regions = ['A222_1','A222_2','A222_3','A222_4a','A222_4b','A222_5a', 'A222_5b', 
    'A223_1','A223_2','A223_3','A223_4b','A223_5a', 'A223_5b', 'bkg', 'werner']
areas = {'A222_1':1*3.14,'A222_2':(2.5**2-1)*3.14,'A222_3':(3.5**2-2.5**2)*3.14,'A222_4a':(5**2-3.5**2)*3.14,
'A222_4b':(5**2-3.5**2)*3.14*65/360,'A222_5a':(7**2-5**2)*3.14*30/360, 'A222_5b':(7**2-5**2)*3.14*30/360, 
'A223_1':1*3.14,'A223_2':(2**2-1)*3.14*300/360,'A223_3':(3**2-2**2)*3.14*300/360,
'A223_4b':(5.5**2-3**2)*3.14*85/360,'A223_5a':(8**2-5.5**2)*3.14*47/360, 'A223_5b':(8**2-5.5**2)*3.14*20/360, 'bkg': 2**2*3.14, 'werner': 1.6**2*3.14}
src_idxes = {'A222_1':[14],'A222_2':[5,6,93],'A222_3':[3,4,96,84,16],
    'A222_4a':[11],'A222_4b':'none','A222_5a':[12,13], 'A222_5b':[23,25,28,31], 
    'A223_1':[64],'A223_2':[59,71],'A223_3':[69,51,42,43,44,58],'A223_4b':[79,73,72,74,78,61,53],'A223_5a':'none', 'A223_5b':[62,49], 'bkg':[7], 'werner':[26, 20, ]}
half_src_idxes = {'A223_4b':[66],'A222_1':[5,6,96],'A222_5a':[19,8], 'A222_4a':[4], 'A222_2':[5,6,96,4], 'A222_3':[4], 'bkg':[8], }

In [ ]:
def suzaku_norm(flux, area):
    return flux/area*400*3.14/5.166e-9
flux_err = np.max((fhi-f, f-flo), axis=0)
file = open(f'/Users/eusracenorth/Documents/suzaku_A222-223/suzaku_data/spectras_230307/pointsrcflux.csv', 'w')
file.write('region, flux, flux_err\n')
for reg in regions:
    src_idx = src_idxes[reg]
    if src_idx=='none':
        ps_flux = 0
        ps_err = 0 
    else:
        src_idx = np.array(src_idx)-1
        ps_flux = f[src_idx].sum()
        ps_err = np.sqrt((flux_err[src_idx]**2).sum())
    if reg in half_src_idxes.keys():
        half_src_idx = np.array(half_src_idxes[reg])-1
        half_srcflux = 0.5 * f[half_src_idx].sum()
        half_err = 0.5 * (flux_err[half_src_idx]**2).sum()**0.5
        sum_flux = half_srcflux + ps_flux
        sum_err = (half_err**2 + ps_err**2).sum()**0.5        
    else:
        half_srcflux = 0
        half_err = 0
        sum_flux = ps_flux
        sum_err = ps_err
    file.write(f'{reg}, {sum_flux}, {sum_err}\n')
    print(reg, half_srcflux, half_err, ps_flux, ps_err, sum_flux, sum_err, suzaku_norm(sum_flux,areas[reg]), suzaku_norm(sum_err,areas[reg]))
file.close()

read rui xmm flux table

In [ ]:
src_idxes = {'A222_1':[14,95],'A222_2':[5,6,93],'A222_3':[3,4,96,84,16],
    'A222_4a':[11],'A222_4b':'none','A222_5a':[12,13], 'A222_5b':[23,25,28,31], 
    'A223_1':[64],'A223_2':[59,71],'A223_3':[69,51,42,43,44,58],'A223_4b':[79,73,72,74,78,61,53],'A223_5a':'none', 'A223_5b':[62,49], 'bkg':[0], 'werner':[26, 20, ]}

In [ ]:
path = '/Users/eusracenorth/Documents/suzaku_A222-223/xmm_data/rui'
hdul = fits.open(f'{path}/srclist_sum_ex_extend_byeye.fits')
data = hdul[1].data
trans = 1/1.2
# where_lst = [(data['OBS_ID']=='0502020101'), (data['OBS_ID']=='0502020201'), (data['OBS_ID']!='0502020101') & (data['OBS_ID']!='0502020201')]
# labels = ['101', '201', 'stack']
where_lst = [(data['OBS_ID']!='0502020101') & (data['OBS_ID']!='0502020201')]
labels = ['stack']
for i, where in enumerate(where_lst):
    flux = (np.array(data["EP_5_FLUX"][where] ) + np.array(data["EP_4_FLUX"][where] ) )* trans
    f_PN = (np.array(data["PN_5_FLUX"][where] ) + np.array(data["PN_4_FLUX"][where] ) )* trans
    f_MOS1 = (np.array(data["M1_5_FLUX"][where] ) + np.array(data["M1_4_FLUX"][where] ) )*trans
    f_MOS2 = (np.array(data["M2_5_FLUX"][where] ) + np.array(data["M2_4_FLUX"][where] ) )*trans
    f = flux
f[f<5e-15] = np.nan
